#  <span style="color:pink">US HOLIDAYS</span>
<span style="color:olive">**2024 Tidytuesday  Week 25** </span>

# Setup

### Libraries

In [61]:
import os
import re
import math
import datetime
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from great_tables import GT, md, html
from great_tables import GT, from_column, style, loc

from matplotlib import rcParams
from matplotlib import font_manager

### Fonts and colors

In [2]:
font_dirs =  ['/Library/Fonts']
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

#rcParams['font.family'] = 'Roboto'

### Data

In [38]:
url_data = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2024/2024-06-18/federal_holidays.csv'
dfr = pd.read_csv(url_data)

# EDA

### Overview

In [6]:
dfr.head()

,date,date_definition,official_name,year_established,date_established,details
0,January 1,fixed date,New Year's Day,1870,1870-06-28,Celebrates the beginning of the Gregorian cale...
1,January 15–21,3rd monday,"Birthday of Martin Luther King, Jr.",1983,1983-11-02,"Honors Dr. Martin Luther King Jr., a civil rig..."
2,February 15–21,3rd monday,Washington's Birthday,1879,NaN,"Honors George Washington, Founding Father, com..."
3,May 25–31,last monday,Memorial Day,1868,NaN,Honors U.S. military personnel who have fought...
4,June 19,fixed date,Juneteenth National Independence Day,2021,2021-06-17,Commemorates the emancipation of enslaved peop...


In [39]:
dfr.columns

Index(['date', 'date_definition', 'official_name', 'year_established',
       'date_established', 'details'],
      dtype='object')

### Renaming columns

In [40]:
dfr.rename(
    columns= {
        'official_name':'holiday', 
        'year_established':'year', 
        'date_definition':'definition',
        'date_established':'established'
        },
    inplace=True)

In [41]:
dfr.head(3)

,date,definition,holiday,year,established,details
0,January 1,fixed date,New Year's Day,1870,1870-06-28,Celebrates the beginning of the Gregorian cale...
1,January 15–21,3rd monday,"Birthday of Martin Luther King, Jr.",1983,1983-11-02,"Honors Dr. Martin Luther King Jr., a civil rig..."
2,February 15–21,3rd monday,Washington's Birthday,1879,NaN,"Honors George Washington, Founding Father, com..."


### Adding and droping columns

- Create a categorical `fixed` column
- Create a `month` column
- Drop `definition` and `date` columns

In [42]:
dfr['fixed'] = np.where(dfr['definition'].str.contains('fixed'), 'Yes', 'No')
dfr.head(3)

,date,definition,holiday,year,established,details,fixed
0,January 1,fixed date,New Year's Day,1870,1870-06-28,Celebrates the beginning of the Gregorian cale...,Yes
1,January 15–21,3rd monday,"Birthday of Martin Luther King, Jr.",1983,1983-11-02,"Honors Dr. Martin Luther King Jr., a civil rig...",No
2,February 15–21,3rd monday,Washington's Birthday,1879,NaN,"Honors George Washington, Founding Father, com...",No


In [43]:
re.sub('[0-9]', '', 'Hello World 2024!')
dfr['date'].str.replace('[\d+]|\s|', '', regex=True).apply(lambda x: x.replace('–', ''));
# need to copy `dash` in column instead of using dash key.

In [44]:
dfr['month'] = dfr['date'].str.replace('[\d+]|\s|–', '', regex=True)
dfs = dfr.drop(['date', 'definition', 'established'], axis=1).copy()
dfs.columns

Index(['holiday', 'year', 'details', 'fixed', 'month'], dtype='object')

### Reordering columns and upper case

In [55]:
df = dfs[['holiday', 'month', 'year', 'fixed', 'details' ]]
df.head(3)

,holiday,month,year,fixed,details
0,New Year's Day,January,1870,Yes,Celebrates the beginning of the Gregorian cale...
1,"Birthday of Martin Luther King, Jr.",January,1983,No,"Honors Dr. Martin Luther King Jr., a civil rig..."
2,Washington's Birthday,February,1879,No,"Honors George Washington, Founding Father, com..."


In [56]:
df.columns = map(lambda x: str(x).upper(), df.columns)
df.head()

,HOLIDAY,MONTH,YEAR,FIXED,DETAILS
0,New Year's Day,January,1870,Yes,Celebrates the beginning of the Gregorian cale...
1,"Birthday of Martin Luther King, Jr.",January,1983,No,"Honors Dr. Martin Luther King Jr., a civil rig..."
2,Washington's Birthday,February,1879,No,"Honors George Washington, Founding Father, com..."
3,Memorial Day,May,1868,No,Honors U.S. military personnel who have fought...
4,Juneteenth National Independence Day,June,2021,Yes,Commemorates the emancipation of enslaved peop...


## Tables

In [57]:
df

,HOLIDAY,MONTH,YEAR,FIXED,DETAILS
0,New Year's Day,January,1870,Yes,Celebrates the beginning of the Gregorian cale...
1,"Birthday of Martin Luther King, Jr.",January,1983,No,"Honors Dr. Martin Luther King Jr., a civil rig..."
2,Washington's Birthday,February,1879,No,"Honors George Washington, Founding Father, com..."
3,Memorial Day,May,1868,No,Honors U.S. military personnel who have fought...
4,Juneteenth National Independence Day,June,2021,Yes,Commemorates the emancipation of enslaved peop...
5,Independence Day,July,1870,Yes,Celebrates the 1776 adoption of the Declaratio...
6,Labor Day,September,1894,No,Honors and recognizes the American labor movem...
7,Columbus Day,October,1968,No,"Honors Christopher Columbus, whose exploration..."
8,Veterans Day,November,1938,Yes,Honors all veterans of the United States armed...
9,Thanksgiving Day,November,1941,No,Traditionally celebrates the giving of thanks ...


In [68]:
(
    GT(df, rowname_col = "HOLIDAY")
    .tab_header(
        title="US HOLIDAYS",
        subtitle="Current Holidays in United States"
    )
    .tab_stubhead(label="NAMES"
    )
)

GT(_tbl_data=                                 HOLIDAY      MONTH  YEAR FIXED  \
0                         New Year's Day    January  1870   Yes   
1    Birthday of Martin Luther King, Jr.    January  1983    No   
2                  Washington's Birthday   February  1879    No   
3                           Memorial Day        May  1868    No   
4   Juneteenth National Independence Day       June  2021   Yes   
5                       Independence Day       July  1870   Yes   
6                              Labor Day  September  1894    No   
7                           Columbus Day    October  1968    No   
8                           Veterans Day   November  1938   Yes   
9                       Thanksgiving Day   November  1941    No   
10                         Christmas Day   December  1870   Yes   

                                              DETAILS  
0   Celebrates the beginning of the Gregorian cale...  
1   Honors Dr. Martin Luther King Jr., a civil rig...  
2   Honors George Washington, Founding Father, com...  
3   Honors U.S. military personnel who have fought...  
4   Commemorates the emancipation of enslaved peop...  
5   Celebrates the 1776 adoption of the Declaratio...  
6   Honors and recognizes the American labor movem...  
7   Honors Christopher Columbus, whose exploration...  
8   Honors all veterans of the United States armed...  
9   Traditionally celebrates the giving of thanks ...  
10  The most widely celebrated holiday of the Chri...  , _body=<great_tables._gt_data.Body object at 0x12baca2d0>, _boxhead=Boxhead([ColInfo(var='HOLIDAY', type=<ColInfoTypeEnum.stub: 2>, column_label='HOLIDAY', column_align='left', column_width=None), ColInfo(var='MONTH', type=<ColInfoTypeEnum.default: 1>, column_label='MONTH', column_align='left', column_width=None), ColInfo(var='YEAR', type=<ColInfoTypeEnum.default: 1>, column_label='YEAR', column_align='right', column_width=None), ColInfo(var='FIXED', type=<ColInfoTypeEnum.default: 1>, column_label='FIXED', column_align='left', column_width=None), ColInfo(var='DETAILS', type=<ColInfoTypeEnum.default: 1>, column_label='DETAILS', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x12bd7e5d0>, _spanners=Spanners([]), _heading=Heading(title='US HOLIDAYS', subtitle='Current Holidays in United States', preheader=None), _stubhead='NAMES', _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x12bd7d050>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), 

In [102]:
(
    GT(df)
    .tab_header(
        title="US HOLIDAYS",
        subtitle="Current Holidays in United States"
    )
    .tab_source_note(
        source_note=md("Source: TidyTuesday | **Week25**")
    )
    .tab_source_note(
        source_note=md("*https://github.com/rfordatascience/tidytuesday/*")
    )
    .tab_style(
        style=style.text(color="dodgerblue", font="Times New Roman", weight="bold"),
        locations=loc.body(columns="HOLIDAY")
    )
    .tab_style(
        style=style.fill(color='cyan'),
        locations=loc.body("HOLIDAY")
    )
    .tab_style(
        style=style.text(color='grey', size = 2),
        locations=loc.body("DETAILS")
    )
    .cols_width(cases={
        "HOLIDAY": "100px",
        "DETAILS": "600px"}
    )
    .tab_options(
        heading_background_color="orchid",

        column_labels_border_top_color="#9400D3",
        column_labels_border_top_style="solid",
        column_labels_border_top_width="5px",

        table_body_hlines_color="grey",
        table_body_hlines_style="dashed",
        table_body_hlines_width="1px",

        table_background_color="wheat"


    )
)



GT(_tbl_data=                                 HOLIDAY      MONTH  YEAR FIXED  \
0                         New Year's Day    January  1870   Yes   
1    Birthday of Martin Luther King, Jr.    January  1983    No   
2                  Washington's Birthday   February  1879    No   
3                           Memorial Day        May  1868    No   
4   Juneteenth National Independence Day       June  2021   Yes   
5                       Independence Day       July  1870   Yes   
6                              Labor Day  September  1894    No   
7                           Columbus Day    October  1968    No   
8                           Veterans Day   November  1938   Yes   
9                       Thanksgiving Day   November  1941    No   
10                         Christmas Day   December  1870   Yes   

                                              DETAILS  
0   Celebrates the beginning of the Gregorian cale...  
1   Honors Dr. Martin Luther King Jr., a civil rig...  
2   Honors George Washington, Founding Father, com...  
3   Honors U.S. military personnel who have fought...  
4   Commemorates the emancipation of enslaved peop...  
5   Celebrates the 1776 adoption of the Declaratio...  
6   Honors and recognizes the American labor movem...  
7   Honors Christopher Columbus, whose exploration...  
8   Honors all veterans of the United States armed...  
9   Traditionally celebrates the giving of thanks ...  
10  The most widely celebrated holiday of the Chri...  , _body=<great_tables._gt_data.Body object at 0x12a7bfed0>, _boxhead=Boxhead([ColInfo(var='HOLIDAY', type=<ColInfoTypeEnum.default: 1>, column_label='HOLIDAY', column_align='left', column_width='100px'), ColInfo(var='MONTH', type=<ColInfoTypeEnum.default: 1>, column_label='MONTH', column_align='left', column_width=None), ColInfo(var='YEAR', type=<ColInfoTypeEnum.default: 1>, column_label='YEAR', column_align='right', column_width=None), ColInfo(var='FIXED', type=<ColInfoTypeEnum.default: 1>, column_label='FIXED', column_align='left', column_width=None), ColInfo(var='DETAILS', type=<ColInfoTypeEnum.default: 1>, column_label='DETAILS', column_align='left', column_width='600px')]), _stub=<great_tables._gt_data.Stub object at 0x12c2a2550>, _spanners=Spanners([]), _heading=Heading(title='US HOLIDAYS', subtitle='Current Holidays in United States', preheader=None), _stubhead=None, _source_notes=[Text(text='Source: TidyTuesday | **Week25**', type='from_markdown'), Text(text='*https://github.com/rfordatascience/tidytuesday/*', type='from_markdown')], _footnotes=[], _styles=[StyleInfo(locname='data', locnum=5, grpname=None, colname='HOLIDAY', rownum=0, colnum=None, styles=[CellStyleText(color='dodgerblue', font='Times New Roman', size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='HOLIDAY', rownum=1, colnum=None, styles=[CellStyleText(color='dodgerblue', font='Times New Roman', size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='HOLIDAY', rownum=2, colnum=None, styles=[CellStyleText(color='dodgerblue', font='Times New Roman', size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='HOLIDAY', rownum=3, colnum=None, styles=[CellStyleText(color='dodgerblue', font='Times New Roman', size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='HOLIDAY', rownum=4, colnum=None, styles=[CellStyleText(color='dodgerblue', font='Times New Roman', size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=Non